In [3]:
import gym
import random 
import numpy as np
import time

from gym.envs.registration import register

In [4]:
#We've used OOPS here

In [5]:
#register(
#    id='FrozenLakeNoSlip-v0',
#    entry_point='gym.envs.toy_text:FrozenLakeEnv',
#    kwargs={'map_name' : '4x4','is_slippery':False },
#    max_episode_steps=100,
#    reward_threshold=0.78, # optimum = .8196
#)

In [6]:
#env_name = "FrozenLakeNoSlip-v0"
#env = gym.make(env_name)

In [7]:
#env.observation_space

In [8]:
#env.action_space

# Reinforcement / Q Learning using Keras - We would've used tensoflow 1.0 but tensorflow 2.0 is out now,
# and it's syntax is quite similiar to that of Keras

# DQN - Deep Q Networks

In [9]:
import random
import gym
import numpy as np


# We'llSkipping making a Q Table and instead we're just making the network as it should have been 

In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam


Using TensorFlow backend.


# Since big games have a lot of possibilities at any point of time...
# So we use Exp - Replay to learn and win the game faster that ever...

# We'll use Double Ended Queue - we call it as "Dequeue" here

In [11]:
from collections import deque

In [ ]:
class DQNAgent:
    
    def __init__(self,state_size,action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = 0.95 # discount factor
        self.epsilon = 1.0
        self.epsilon_decay = 0.995 # Kitni Value Bachni Chahiye
        self.epsilon_min = 0.01
        self.learning_rate =0.01  #we can call it alpha as well
        self.memory = deque(maxlen = 2000)
        self.model = self.buil_model()
        
    def buil_model(self):
        model = Sequential()
        model.add(Dense(24,input_dim = self.state_size, activation = 'relu'))
        model.add(Dense(24,activation = 'relu'))
        model.add(Dense(self.action_size,activation = 'linear'))
        model.compile(loss = 'mse',optimizer = Adam(lr = self.learning_rate)) # mse = mean Squared Error
        return model
    
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    def remember(self, state, action , reward, next_state, done):
        self.memory.append((state,action,reward,next_state,done)) # Append Karne ke liye tuple pass kar diya taaki saari values ek hi baari  
        # mien paas ho jaaye otherwise .append() mien ek time pe ek hi value daal paate hai
        
    def replay(self,batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for (state,action,reward,next_state,done) in minibatch:
            target = reward
            if not done:
                target += self.gamma*np.amax(self.model.predict(next_state))
            target_matrix = self.model.predict(state)
            target_matrix[0][action] = target
            self.model.fit(state , target_matrix,epochs  = 1,verbose = 0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            

In [ ]:
# Here our policy is based on what the model has returned

In [17]:
episodes = 100 # 100

env = gym.make("CartPole-v1")
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size,action_size)
done = False
batch_size = 32

for e in range(episodes):
    state = env.reset()
    state = np.reshape(state,[1,state_size])
    for time in range(500):
        # env.render()
        action = agent.act(state)
        next_state,reward,done, _ = env.step(action)
        # is the pole falls -ve reward milega usko
        reward = reward if not done else -10
        next_state = np.reshape(next_state,[1,state_size])
        agent.remember(state,action,reward,next_state, done)
        state = next_state
        if done :
            print(f"Episodes:{e}/{episodes} Time:{time} Exploration:{agent.epsilon}")
            break
        if len(agent.memory)>batch_size:
            agent.replay(batch_size)
    
#    env.close()    

Episodes:0/100 Time:20 Exploration:1.0
Episodes:1/100 Time:32 Exploration:0.9000874278732445
Episodes:2/100 Time:24 Exploration:0.798065677681905
Episodes:3/100 Time:19 Exploration:0.7255664080186093
Episodes:4/100 Time:23 Exploration:0.6465587967553006
Episodes:5/100 Time:13 Exploration:0.6057704364907278
Episodes:6/100 Time:26 Exploration:0.531750826943791
Episodes:7/100 Time:9 Exploration:0.5082950737585841
Episodes:8/100 Time:9 Exploration:0.4858739637363176
Episodes:9/100 Time:8 Exploration:0.46677573701590436
Episodes:10/100 Time:16 Exploration:0.43080185560799106
Episodes:11/100 Time:11 Exploration:0.40769130904675194
Episodes:12/100 Time:12 Exploration:0.38389143477919885
Episodes:13/100 Time:9 Exploration:0.3669578217261671
Episodes:14/100 Time:69 Exploration:0.25966219297210513
Episodes:15/100 Time:26 Exploration:0.22793384675362674
Episodes:16/100 Time:35 Exploration:0.1912566947289212
Episodes:17/100 Time:35 Exploration:0.16048131420416054
Episodes:18/100 Time:40 Exploratio

KeyboardInterrupt: 